In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,50584
2,Huelva,Confirmados PDIA 14 días,2637
3,Huelva,Tasa PDIA 14 días,"513,8648011380244"
4,Huelva,Confirmados PDIA 7 días,1625
5,Huelva,Tasa PDIA 7 dias,"316,6591967574098"
6,Huelva,Total Confirmados,50800
7,Huelva,Curados,46763
8,Huelva,Fallecidos,425


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  50584.0


/tmp/ipykernel_60130/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  14200.0


/tmp/ipykernel_60130/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_60130/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_60130/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_60130/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 264 personas en los últimos 7 días 

Un positivo PCR cada 164 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,50584.0,1625.0,2637.0,513170.0,316.659197,513.864801,363.0
Huelva-Costa,29806.0,916.0,1480.0,289548.0,316.355147,511.141503,204.0
Huelva (capital),14200.0,545.0,875.0,143837.0,378.901117,608.327482,134.0
Condado-Campiña,15664.0,421.0,701.0,156231.0,269.472768,448.694561,103.0
Sierra de Huelva-Andévalo Central,4682.0,285.0,452.0,67391.0,422.905136,670.712706,54.0
Lepe,3163.0,62.0,95.0,27880.0,222.381636,340.746055,17.0
Aljaraque,1845.0,73.0,130.0,21474.0,339.945981,605.383254,17.0
Bollullos Par del Condado,1503.0,74.0,117.0,14387.0,514.353236,813.234170,12.0
Ayamonte,2219.0,72.0,89.0,21104.0,341.167551,421.721001,12.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puerto Moral,18.0,4.0,13.0,278.0,1438.848921,4676.258993,2.0
Cumbres Mayores,184.0,18.0,45.0,1749.0,1029.159520,2572.898799,6.0
Aracena,862.0,113.0,180.0,8255.0,1368.867353,2180.496669,12.0
Marines (Los),22.0,7.0,7.0,399.0,1754.385965,1754.385965,1.0
Corteconcepción,56.0,5.0,7.0,536.0,932.835821,1305.970149,1.0
Villanueva de las Cruces,26.0,2.0,5.0,387.0,516.795866,1291.989664,1.0
Fuenteheridos,23.0,5.0,8.0,709.0,705.218618,1128.349788,2.0
Almonaster la Real,108.0,13.0,15.0,1788.0,727.069351,838.926174,3.0
Bollullos Par del Condado,1503.0,74.0,117.0,14387.0,514.353236,813.234170,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Puerto Moral,18.0,4.0,13.0,278.0,1438.848921,4676.258993,2.0,0.307692
Villablanca,288.0,1.0,3.0,2885.0,34.662045,103.986135,0.0,0.333333
Cumbres Mayores,184.0,18.0,45.0,1749.0,1029.159520,2572.898799,6.0,0.400000
Villanueva de las Cruces,26.0,2.0,5.0,387.0,516.795866,1291.989664,1.0,0.400000
Almonte,2494.0,45.0,107.0,24507.0,183.621006,436.609948,8.0,0.420561
Palma del Condado (La),1525.0,15.0,35.0,10801.0,138.876030,324.044070,3.0,0.428571
Calañas,260.0,8.0,17.0,2768.0,289.017341,614.161850,4.0,0.470588
Isla Cristina,3255.0,26.0,55.0,21393.0,121.535082,257.093442,7.0,0.472727
Jabugo,90.0,1.0,2.0,2260.0,44.247788,88.495575,0.0,0.500000
